# zk-Autograd: Provable Training Demo

This notebook demonstrates the core features of **zk-Autograd**, a framework for generating Zero-Knowledge (ZK) proofs of training steps for PyTorch models. It showcases:

1.  **Setup**: Compiling ZK circuits for the optimizer (Adam).
2.  **Training**: Running a provable training loop with a toy model.
3.  **Audit Log**: Inspecting the cryptographic audit log and Merkle tree.
4.  **Verification**: Verifying the generated ZK proofs.
5.  **Advanced Features**: Triton kernel integration and proof chunking.
6.  **Decentralized Storage**: Packaging artifacts into a torrent bundle.
7.  **Security**: Fuzzing the verifier to ensure robustness.


## 1. Setup and Installation

First, we install the package in editable mode and verify that the necessary dependencies (EZKL, PyTorch) are available.

In [ ]:
!pip install -e .
import torch
import ezkl
print(f"PyTorch version: {torch.__version__}")
print(f"EZKL version: {ezkl.__version__}")

### Circuit Compilation

We compile the ZK circuit for the Adam optimizer. This generates the proving key (`pk.key`), verification key (`vk.key`), and the compiled circuit (`compiled.ezkl`).

In [ ]:
!zk-setup-zk --circuit adam --dim 128 --out prover/keys

## 2. Provable Training Loop

We now run a short training session using `zk_autograd.trainer`. This will:
- Train a `TinyMLP` model on synthetic data.
- Capture execution traces (witnesses) for each optimization step.
- Generate ZK proofs (if a prover is available/configured).
- Log everything to a tamper-evident audit log.

In [ ]:
from zk_autograd.trainer import run_demo_train

# Run 5 steps of training
run_dir = run_demo_train(steps=5, prover_url="http://127.0.0.1:8000")
print(f"Training complete. Artifacts stored in: {run_dir}")

## 3. Inspecting the Audit Log

The training run produces a `steps.jsonl` file containing the metadata and proof hashes for each step. These hashes are aggregated into a Merkle tree to ensure integrity.

In [ ]:
import os, json

log_path = os.path.join(run_dir, "steps.jsonl")
with open(log_path, "r") as f:
    for line in f:
        step = json.loads(line)
        print(f"Step {step['step_idx']}: Loss={step['loss']:.4f}, Proof Hash={step['proof_hash'][:16]}...")

merkle_root_path = os.path.join(run_dir, "merkle_root.txt")
if os.path.exists(merkle_root_path):
    print(f"\nMerkle Root: {open(merkle_root_path).read().strip()}")

## 4. Verification

We can randomly sample steps from the run and verify their ZK proofs against the committed public inputs (weights, gradients, hyperparameters).

In [ ]:
from verifier.verify_steps import verify_random_steps

# Verify 3 random steps from the run
verify_random_steps(run_dir, k=3, key_dir="prover/keys")

## 5. Advanced Features

### Triton Kernels
zk-Autograd supports custom Triton kernels for high-performance fused Adam updates on GPU. This ensures that the execution trace matches the ZK circuit logic exactly.

In [ ]:
from zk_autograd.triton_kernels import available

if available():
    print("Triton/CUDA is available! Using fused kernels for training.")
else:
    print("Triton/CUDA not detected. Falling back to pure PyTorch implementation.")

### Proof Chunking
For large models, generating a single monolithic proof is infeasible. We support splitting the model update into smaller chunks, proving them in parallel, and aggregating the results.

In [ ]:
from zk_autograd.splitting import plan_split

# Example: Plan a split of the Adam step into 4 chunks
plan = plan_split("prover/keys/adam_step.onnx", chunks=4, out_dir="prover/keys/chunks")
print("Split Plan:", json.dumps(plan, indent=2))

## 6. Decentralized Storage (Torrents)

To ensure availability and censorship resistance, we package the training artifacts (proofs, logs, models) into a torrent bundle. This allows anyone to seed and download the verifiable training run.

In [ ]:
from zk_autograd.torrents import create_toy_torrent_bundle

# Create a torrent bundle for the run directory
manifest = create_toy_torrent_bundle(run_dir, out_dir="public/torrents")
print(f"Torrent created: {manifest['magnet']}")
print("Files included:", [f['path'] for f in manifest['files']])

### Supply Chain Security

In a production environment, the Docker images used for the prover and trainer are signed using **Sigstore (Cosign)**, and an SBOM is generated to ensure supply chain integrity. This prevents tampering with the software artifacts before they are deployed to the TEE.

## 7. Security: Verifier Fuzzing

Security is paramount. We use property-based fuzzing (Hypothesis) to ensure the verifier is robust against malformed inputs.

In [ ]:
!pytest tests/test_fuzz_verifier.py